Note that the output of all cells has been cleared in order to reduce the file size. If you wish to reproduce the results in the report, please re-run all cells.

### Format the data to work with Prophet API

In [ ]:
import pandas as pd
from prophet import Prophet
import numpy as np
df = pd.read_csv('data/projectDataClean.csv')
df.head()

In [ ]:
df_train = pd.read_csv('data/train.csv')
df_val = pd.read_csv('data/val.csv')
df_test = pd.read_csv('data/test.csv')

In [ ]:
df_train = df_train.rename(columns = {'time' : 'ds'})
df_train = df_train.rename(columns = {'t_sla': 'y'})
df_val = df_val.rename(columns = {'time' : 'ds'}) 
df_val = df_val.rename(columns = {'t_sla': 'y'})
df_test = df_test.rename(columns = {'time' : 'ds'}) 
df_test = df_test.rename(columns = {'t_sla': 'y'})
df = df.rename(columns = {'t_sla': 'y'})
df = df.rename(columns = {'t_sla': 'y'})

In [ ]:
train_val = pd.concat([df_train,df_val])

In [ ]:
#Specify Validation Period 
val_start = train_val.index[train_val['ds'] == df_val.ds[0]].tolist()[0]
val_start

### Make a rolling prediction every 1 hour on the validation data, letting the model firstly fit on all the modeling data

In [ ]:
from prophet.diagnostics import cross_validation
m = Prophet()
m.fit(train_val)
df_cv = cross_validation(m, initial = str(val_start) + " hour" ,period = '1 hour', horizon = '1 hour')

### Check the variance of prediction residual

In [ ]:
import numpy as np
error = df_cv.y - df_cv.yhat
np.var(error)

### Make a rolling prediction every one hour on the modeling data

In [ ]:
from prophet.diagnostics import cross_validation
m = Prophet()
m.fit(df_train)
df_cv = cross_validation(m ,period = '1 hour', horizon = '1 hour')

### Check the variance of prediction residual

In [ ]:
error = df_cv.y - df_cv.yhat
np.var(error)

### Do the same with a 9 step forecasting horizon

In [ ]:
m = Prophet()
m.fit(train_val)
df_cv = cross_validation(m, initial = str(val_start) + " hour" ,period = '1 hour', horizon = '9 hour')

In [ ]:
error = df_cv.y - df_cv.yhat
np.var(error)


In [ ]:
m = Prophet()
m.fit(df_train)
df_cv = cross_validation(m ,period = '1 hour', horizon = '9 hour')

In [ ]:
error = df_cv.y - df_cv.yhat
np.var(error)

### Make some  plots
Note that this is not what is done when making a rolling prediction on all data this would be a lengt(data_set)-step prediction. This is why we use the crossvalidate functionality in the prophet API. These cells are only for getting an intuition for how it works.

In [ ]:
fig1 = m.plot(pred, ylabel ="°C")
plt.title("Validation")
plt.savefig('prophet_val.png')

In [ ]:
np.var(df_val.y - pred.yhat)

In [ ]:
fig2 = m.plot_components(pred)
plt.savefig('figs/prophet_comp_val.png')

In [ ]:
pred_test = m.predict(df_test)

In [ ]:
fig2 = m.plot(pred_test, ylabel ="°C")
plt.title("Test")
plt.savefig('figs/prophet_test.png')

In [ ]:
fig2 = m.plot_components(pred)
plt.savefig('figs/prophet_comp_test.png')